# Analysis of Elevation Profiles of Hiking Trails

## Introduction
This project focuses on the analysis of elevation profiles of hiking trails. The data sources used are hiking trails from OpenStreetMap (OSM) and elevation data from the DMR 5G dataset obtained from the Czech Office for Surveying, Mapping and Cadastre (ČÚZK).

First, the necessary libraries are imported.

In [3]:
# Import standard libraries
import os
import warnings

# Import libraries for data and geospatial data processing
import requests
import geopandas as gpd
import rasterio
from rasterio.plot import show
from shapely.geometry import LineString

# Import libraries for visualization
import matplotlib.pyplot as plt

# Import libraries for working with OSM data
import osmapi

Download of relation using OsmApi

In [5]:
# Function to download a relation by its ID
def download_relation(relation_id):
    try:
        with osmapi.OsmApi() as api:
            return api.RelationGet(relation_id)
    except osmapi.errors.ElementDeletedApiError:
        print(f"Relation with ID {relation_id} has been deleted.")
        return None

Example Usage

In [10]:
# Example ID
relation_id = 222947
relation = download_relation(relation_id)
print(relation)

{'id': 222947, 'visible': True, 'version': 117, 'changeset': 156228438, 'timestamp': datetime.datetime(2024, 9, 5, 12, 18, 58), 'user': '8err', 'uid': 16361887, 'tag': {'complete:guideposts': 'no', 'complete:trail': 'yes', 'destinations': 'Plzeň-Košutka;Krkavec;Chotíkov;Vranov;Stříbro;Svojšín (zámek);Brod nad Tichou;Chodová Planá', 'kct_red': 'major', 'network': 'rwn', 'operator': 'cz:KČT', 'osmc:symbol': 'red:white:red_bar', 'ref': '0201', 'route': 'hiking', 'type': 'route'}, 'member': [{'type': 'node', 'ref': 2277611545, 'role': 'guidepost'}, {'type': 'node', 'ref': 477410997, 'role': 'guidepost'}, {'type': 'node', 'ref': 5225076924, 'role': 'guidepost'}, {'type': 'node', 'ref': 4933227822, 'role': 'guidepost'}, {'type': 'node', 'ref': 4933227821, 'role': 'guidepost'}, {'type': 'node', 'ref': 444824086, 'role': 'guidepost'}, {'type': 'node', 'ref': 313052484, 'role': 'guidepost'}, {'type': 'node', 'ref': 444802057, 'role': 'guidepost'}, {'type': 'node', 'ref': 673717121, 'role': 'gui